In [41]:
import Secreit
from matplotlib import pyplot as plt
import glob
import numpy as np
import os
from keras.applications import VGG16
from keras.callbacks import ModelCheckpoint
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing import image

# Training
## Step1: Split images and save them
## Step2: Training the last 2 layers
## Step3: Training 100 epochs

In [ ]:
# Step1
for dataset in ["train", "validation", "test"]:
    for satge in ["D", "E", "P"]:
        for path in glob.glob("*/data/"+dataset+"/"+stage+"/*.png"):
            img_id=os.path.basename(path)[:-4]
            img=image.load_img(path, target_size=(480, 640) )
            img1=image.img_to_array(img)[:240, :320]
            img2=image.img_to_array(img)[240:480, :320]
            img3=image.img_to_array(img)[:240, 320:640]
            img4=image.img_to_array(img)[240:480, 320:640]
            
            image.save_img("*/data/split_"+dataset+"/"+stage+"/"+img_id+"_1.png",  img1)
            image.save_img("*/data/split_"+dataset+"/"+stage+"/"+img_id+"_2.png",  img2)
            image.save_img("*/data/split_"+dataset+"/"+stage+"/"+img_id+"_3.png",  img3)
            image.save_img("*/data/split_"+dataset+"/"+stage+"/"+img_id+"_4.png",  img4)

In [2]:
# Step 2
trainD=glob.glob("*/data/split_train/D/*.png")
trainE= glob.glob("*/data/split_train/E/*.png")
trainP= glob.glob("*/data/split_train/P/*.png")

valD=glob.glob("*/data/split_validation/D/*.png")
valE=glob.glob("*/data/split_validation/E/*.png")
valP=glob.glob("*/data/split_validation/P/*.png")

In [3]:
train_datagen =Secreit.TrainGenerator(trainD, trainE, trainP)
val_datagen =Secreit.ValidationGenerator(valD, valE, valP)

In [4]:
conv_base=VGG16(weights='imagenet',
include_top=False,
input_shape=(240, 320, 3))
conv_base.trainable=False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(500, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(3, activation="softmax"))
model.compile(loss='categorical_hinge',
optimizer=optimizers.Nadam(lr=2e-5),
metrics=['acc'])

callbacks = list()
callbacks.append(ModelCheckpoint(filepath="*/last_layer_lerning/weight_epoch{epoch:02d}-{val_loss:.2f}.h5"))

In [ ]:
history = model.fit_generator(
    generator=train_datagen.flow_from_directory(),
    steps_per_epoch=58,
    validation_data=val_datagen.flow_from_directory(),
    validation_steps=20,
    callbacks=callbacks,
    epochs=50)  

In [ ]:
# Step 3
## Lord the best parameter from */last_layer_lerning directori

model.load_weights("*h5")
for l in model.layers:
    l.trainable=True

callbacks = list()
callbacks.append(ModelCheckpoint(filepath="*/all_tuning/weight_epoch{epoch:02d}-{val_loss:.2f}.h5"))

history = model.fit_generator(
    generator=train_datagen2.flow_from_directory(),
    steps_per_epoch=58,
    validation_data=test_datagen2.flow_from_directory(),
    validation_steps=20,
    callbacks=callbacks,
    epochs=100)  